## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)

# Spatial Distibution of Low-mass Subhalos

## Motivation

Here, I make a projection plot of the centres of potential of subhalos. I'm particularly looking at the low-mass halos and inspecting the beads-on-a-string structures that appear in the curv-p082 model. Many of these beads likely do not represent physical halos, since they are very low in mass, but are rather traces of the non-physical fragmentation of physical filamentary structures, due to numerical noise. 

Thus, we can use these figures to find a kind of upper mass limit for the subhalos that we trust to be non-numerical.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['plain-LCDM', 'spec-p082']

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

Set the low-mass threshold. Subhalos with masses above or below this limit are plotted on different subplots.

In [ ]:
lowm = 10

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns}

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    cops = snap.get_subhalos('CentreOfPotential') * units.cm.to(units.Mpc)
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)

    # Split into satellites:    
    m31 = sim_data['M31_identifier']
    mw = sim_data['MW_identifier']
    if distinction == 'by_r':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, m31, mw)
    elif distinction == 'by_gn':
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, m31, mw)
        
    # Compute masking arrays:
    mask_m31 = masks_sat[0]
    mask_mw = masks_sat[1]
    mask_subhalo = np.logical_and(np.arange(mask_m31.size) != snap.index_of_halo(m31[0], m31[1]),
                                  np.arange(mask_m31.size) != snap.index_of_halo(mw[0], mw[1]))
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    
    # Mask low mass subhalos:
    mask_lowm = vmax < lowm
    
    # Prune potential spurious:
    mask_nonzero_vmax = dataset_comp.prune_vmax(snap)
    mask_m31 = mask_m31[mask_nonzero_vmax]
    mask_mw = mask_mw[mask_nonzero_vmax]
    mask_lum = mask_lum[mask_nonzero_vmax]
    mask_dark = mask_dark[mask_nonzero_vmax]
    mask_subhalo = mask_subhalo[mask_nonzero_vmax]

    # Add datasets to dictionary:
    data[name]['LG_centre'] = dataset_comp.compute_LG_centre(snap, m31, mw) \
        * units.cm.to(units.Mpc)
    m31_idx = snap.index_of_halo(m31[0], m31[1])
    data[name]['M31'] = {'COP': cops[m31_idx], 'Vmax': vmax[m31_idx]}
    mw_idx = snap.index_of_halo(mw[0], mw[1])
    data[name]['MW'] = {'COP': cops[mw_idx], 'Vmax': vmax[mw_idx]}
    data[name]['COP'] = cops
    data[name]['Vmax'] = vmax
    
    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Isolated': mask_isol,
        'Subhalos': mask_subhalo,
        'Luminous': mask_lum,
        'Dark': mask_dark,
        'LowMass': mask_lowm
    }

---

## Plot 

Plot subhalos with vmax < lowm in the upper row, and the more massive ones in the lower one. Plot two columns for both simulations.

First, set figure parameters:

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Set marker size limits:
slow = 0.2; smax = 3
mlow = lowm; mmax = 0
smin = 0.05

# Iterate through simulations to find largest value of vmax:
for sim_data in data.values():    
    mmax = max(
        mmax, np.max(sim_data['Vmax'][sim_data['Selections']['Subhalos']])
    )
    
print(smin, slow, smax)
print(mlow, mmax)    

In [ ]:
def mass_to_marker(m):
    b = np.exp(smax)
    return c * np.log(a*m+b)

# Select projection plane:
x = 0; y = 1

In [ ]:
test_x = np.arange(10,100, 0.05)
test_y = mass_to_marker(test_x)
plt.plot(test_x, test_y)

In [ ]:
test_x = np.arange(3, 100, 1)
test_y = mass_to_marker(test_x)
plt.scatter(test_x, test_y, s=test_y)

In [ ]:
# Set marker size limits:
slow = 0.5; smax = 3
mlow = lowm; mmax = 0

# Iterate through simulations to find largest value of vmax:
for sim_data in data.values():    
    mmax = max(
        mmax, np.max(sim_data['Vmax'][sim_data['Selections']['Subhalos']])
    )
    
# def m_to_s(m):
#     b = ((mmax/mlow)**2/3 * slow - smax) / ((mmax/mlow)**2/3 - 1)
#     a = (smax - b) / mmax**(2/3)
#     return a * m**(2/3) + b

def m_to_s(m):
    return slow/mlow**(2/3) * m**(2/3)

In [ ]:
x = np.arange(3, 100, 1)
y = m_to_s(x)
plt.scatter(x, y, s=y)

In [ ]:
def mass_to_marker(m):
    return slow/mlow**(2/3) * m**(2/3)

In [ ]:
def mass_to_marker(m):
    b = ((mmax/mlow)**2/3 * slow - smax) / ((mmax/mlow)**2/3 - 1)
    a = (smax - b) / mmax**(2/3)
    return a * m**(2/3) + b

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 9,
              'xtick.labelsize': 6,
              'ytick.labelsize': 6,
              'legend.fontsize': 8}

# Set marker size limits:
slow = 0.1; smax = 1
mlow = lowm; mmax = 0

# Iterate through simulations to find largest value of vmax:
for sim_data in data.values():    
    mmax = max(
        mmax, np.max(sim_data['Vmax'][sim_data['Selections']['Subhalos']])
    )
  
def mass_to_marker(m):
    return slow/mlow**(2/3) * m**(2/3)

# def mass_to_marker(x):
#     a = (smax - slow) / np.log(mmax/mlow)
#     b = slow - a * np.log(mlow)
#     return a * np.log(x) + b

# Select projection plane:
x = 0; y = 1

In [ ]:
print(mlow, mmax)
print(slow, smax)

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(sharey='all', sharex='all', ncols=len(data), nrows=2,
                         figsize=(4,4))
# plt.subplots_adjust(wspace=0.05)
# plt.subplots_adjust(hspace=0.05)

# Set subfigure box side aspect ratio:
for ax in axes.flatten():
    ax.set_aspect(1)

plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.05)

axes[0,0].set_ylabel('$y [\mathrm{Mpc}]$')
axes[1,0].set_xlabel('$x [\mathrm{Mpc}]$')
axes[1,0].set_ylabel('$y [\mathrm{Mpc}]$')
axes[1,1].set_xlabel('$x [\mathrm{Mpc}]$')
    
# Get average of LG centre across simulations:
centre = np.mean(np.vstack(
    [sim_data['LG_centre'] for sim_data in data.values()]),
                 axis=0)

# Set axis limits:
ax_size = 5
axes[1, 0].set_xlim(centre[x] - ax_size/2, 
                    centre[x] + ax_size/2)
axes[1, 0].set_ylim(centre[y] - ax_size/2, 
                    centre[y] + ax_size/2)

# Iterate through simulations (columns):
for i, (key, sim_data) in enumerate(data.items()):
    
    axes[0, i].set_title(key)
    
    # Plot spurious low-mass subhalos in the upper row:
    mask_dark = np.logical_and(sim_data['Selections']['Dark'],
                          sim_data['Selections']['LowMass'])
    axes[0,i].scatter(sim_data['COP'][mask_dark][:,x], 
                      sim_data['COP'][mask_dark][:,y],
                      c='black',
                      alpha=0.7,
                      s=mass_to_marker(sim_data['Vmax'][mask_dark]))
    
    mask_lum = np.logical_and(sim_data['Selections']['Luminous'],
                              sim_data['Selections']['LowMass'])    
    axes[0,i].scatter(sim_data['COP'][mask_lum][:,x], 
                      sim_data['COP'][mask_lum][:,y],
                      facecolors='none', 
                      edgecolors='blue',
                      alpha=0.7,
                      s=mass_to_marker(sim_data['Vmax'][mask_lum]))

    
    # Plot physically representative more massive halos in the lower row:
    mask_dark = np.logical_and.reduce([sim_data['Selections']['Subhalos'],
                                       sim_data['Selections']['Dark'],
                                       np.logical_not(sim_data['Selections']['LowMass'])])
    axes[1,i].scatter(sim_data['COP'][mask_dark][:,x], 
                      sim_data['COP'][mask_dark][:,y],
                      c='k',
                      alpha=0.7,
                      s=mass_to_marker(sim_data['Vmax'][mask_dark]))

    mask_lum = np.logical_and.reduce([sim_data['Selections']['Subhalos'],
                                      sim_data['Selections']['Luminous'],
                                      np.logical_not(sim_data['Selections']['LowMass'])])    
    axes[1,i].scatter(sim_data['COP'][mask_lum][:,x], 
                      sim_data['COP'][mask_lum][:,y],
                      facecolors='none', 
                      edgecolors='blue',
                      alpha=0.5,
                      s=mass_to_marker(sim_data['Vmax'][mask_lum]))
    #print(mass_to_marker(sim_data['Vmax'][mask_lum]))
    
    # Plot central halo centres in all subplots:    
    axes[0,i].scatter(sim_data['M31']['COP'][x], sim_data['M31']['COP'][y],
                      c='red',
                      s=1)
    axes[0,i].scatter(sim_data['MW']['COP'][x], sim_data['MW']['COP'][y],
                      c='red',
                      s=1)
    axes[1,i].scatter(sim_data['M31']['COP'][x], sim_data['M31']['COP'][y],
                      c='red',
                      s=1)
    axes[1,i].scatter(sim_data['MW']['COP'][x], sim_data['MW']['COP'][y],
                      c='red',
                      s=1)
    
    # Set text size:
#     s = 14
#     axes[0, i].title.set_size(s)
#     axes[1, i].xaxis.label.set_size(s)
#     axes[0, 0].yaxis.label.set_size(s)
#     axes[1, 0].yaxis.label.set_size(s)
    
    # Plot 2Mpc circles (for isolated):
    def circle(centre_x, centre_y, r, n):
        t = 4*np.pi/n * np.arange(n/2)
        x = centre_x + r * np.cos(t)
        y = centre_y + r * np.sin(t)
        return x, y

    lw = 0.3
    x_circ, y_circ = circle(centre[0], centre[1], 2, 10000)
    axes[0,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    axes[1,i].plot(x_circ, y_circ, c='black', linestyle='dashed', linewidth=lw)
    
# Add text box:
axes[0,1].text(0.9, 0.1, "$v_\mathrm{{max}} < {} \mathrm{{km/s}}$".format(lowm),
               transform=axes[0,1].transAxes, ha="right", va="bottom")

plt.subplots_adjust(wspace=0.05)
plt.subplots_adjust(hspace=0.05)
# plt.tight_layout()

### Save the Figure

In [ ]:
# Construct saving location:
filename = 'Satellite_locations_low_mass_LOG'
for name in names:
    filename += '_{}'.format(name)
filename += '.png'
    
home = os.path.abspath(os.path.join('..'))
path = os.path.join(home,'Figures', 'MediumResolution')
filename = os.path.join(path, filename)
    
fig.savefig(filename, dpi=300)